In [1]:
from tkinter import *
from tkinter import filedialog
import os
import tkinter as tk
import pickle
import PIL.Image
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from keras.models import load_model
from  keras.preprocessing import image
from keras.applications.xception import preprocess_input, decode_predictions
from PIL import Image, ImageTk

In [2]:
# Load the model
my_model = tf.keras.models.load_model('xception_tr_learning.h5')

# Load the dog breed list
with open('breed_list.pkl', 'rb') as f:
    dog_breed = pickle.load(f)

In [3]:
# Define a load image function
def load_img():
    global img, image_data
    for img_display in frame.winfo_children():
        img_display.destroy()

    image_data = filedialog.askopenfilename(initialdir="/", title="Choose an image",
                                       filetypes=(("JPG File", "*.jpg"), ("PNG File", "*.png"), ("All Files", "*.*")))
    basewidth = 150 # Processing image for displaying
    img = Image.open(image_data)
    img.thumbnail((400,400))
    img = ImageTk.PhotoImage(img)
    file_name = image_data.split('/')
    panel = tk.Label(frame, text= str(file_name[len(file_name)-1]).upper(), fg="white", bg='grey').pack()
    panel_image = tk.Label(frame, image=img).pack()

In [4]:
# Define a classify the image function
def classify():
    original = Image.open(image_data)
    original = original.resize((299, 299), Image.ANTIALIAS)
    numpy_image = image.img_to_array(original)
    image_batch = np.expand_dims(numpy_image, axis=0)
    processed_image = preprocess_input(image_batch)
    
    predictions = my_model.predict(processed_image)
    label =  dog_breed[np.argmax(predictions)]
    pred = round(np.max(predictions[0]), 2)
    
    spacer1 = tk.Label(frame, text="", bg='grey').pack() # to add a space
    table = tk.Label(frame, text="Dog Breed Prediction", fg="black", font=("Times", 20, "bold"), bg='grey').pack()
    result = tk.Label(frame, text=label, fg="white", font=("Times", 20, "bold"), bg='grey').pack()
    
    table1 = tk.Label(frame, text=("Probability", pred), fg="white", font=("Times", 12, "italic"), bg="grey").pack()

In [5]:
# Define the parameters of the local browser to classify dog images
root = tk.Tk()

root.title('Image Classifier (OpenClassrooms Project - Adib Ouayjan)')
root.resizable(True, True)
tit = tk.Label(root, text="Image Classifier for Dog Breeds", padx=25, pady=6, font=("Times", 22)).pack()

canvas = tk.Canvas(root, height=500, width=700, bg='grey')
canvas.pack()

frame = tk.Frame(root, bg='grey')
frame.place(relwidth=0.8, relheight=0.8, relx=0.1, rely=0.1)

chose_image = tk.Button(root, text='Choose Image',
                        padx=35, pady=10,
                        fg="white", bg="darkblue", font=("Times", 18, "bold"), command=load_img)
chose_image.pack(side=tk.LEFT)

class_image = tk.Button(root, text='Classify Image',
                        padx=35, pady=10,
                        fg="white", bg="darkorange", font=("Times", 18, "bold"),command=classify)
class_image.pack(side=tk.RIGHT)

root.mainloop()